In [11]:
import numpy as np
import random # 시드 고정을 위해
import os # 시드 고정을 위해
import pandas as pd
import joblib

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

import torch
from torch import Tensor,tensor
from torch.utils.data import DataLoader,Dataset
from torch.nn import Module,Sequential

device= 'cuda' if torch.cuda.is_available() else 'cpu'

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

import pymysql
import pandas as pd

conn = pymysql.connect(host='127.0.0.1', user='wodus1530',
                       password='dkvms7255', db='Zigzag')


cursor = conn.cursor()


sql_query = 'SELECT * FROM db'
cursor.execute(sql_query)

result = cursor.fetchall()
column_names = [i[0] for i in cursor.description]


db = pd.DataFrame(result, columns=column_names)

cursor.close()
conn.close()



In [12]:
thick_data_1 = pd.DataFrame(db[db['두께'] != 0]['리뷰'], columns=['리뷰']).reset_index(drop=True)
thick_data_1['target'] = 1

thick_data_2 = pd.DataFrame(db[db['두께'] == 0]['리뷰'], columns=['리뷰']).reset_index(drop=True)
thick_data_2['target'] = 0

thick_data = pd.concat([thick_data_1, thick_data_2], axis=0).reset_index(drop=True)
thick_data.isnull().sum()

리뷰        0
target    0
dtype: int64

In [13]:
from mecab import MeCab
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer

filter=['NNG','MAG','EC','VA','VA+EF','VV+ETM','NNB+JKB','VCP+EC','VCP','MAG+JX','VCN']

def tokenizer(data):
    tokenizer = MeCab()
    
    list = []
    
    for text in tqdm(data["리뷰"]):
        prah = []  
        lst = tokenizer.pos(text)  
        for word, pos in lst:
            if pos in filter:
                prah.append(word)  
        list.append(' '.join(prah))  
    
    return pd.DataFrame({'tokens':list})

DB_list=tokenizer(thick_data)

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=100)
legacy_tfidf = vectorizer.fit_transform(DB_list["tokens"])

joblib.dump(vectorizer,'thick_vectorizer.pkl')

legacy_tfidf=legacy_tfidf.toarray()

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

legacy_tfidf = scaler.fit_transform(legacy_tfidf)

joblib.dump(scaler,'thick_scaler.pkl')

from catboost import CatBoostClassifier


catboost_model = CatBoostClassifier(random_state=42, verbose=0)


catboost_model.fit(legacy_tfidf, thick_data['target'])


feature_importances = catboost_model.feature_importances_

feature_names = vectorizer.get_feature_names_out()


sorted_indices = np.argsort(feature_importances)[::-1]
sorted_feature_importances = feature_importances[sorted_indices]
sorted_feature_names = feature_names[sorted_indices]

100%|██████████| 5135/5135 [00:06<00:00, 838.27it/s] 


In [14]:
top_10_indices = sorted_indices[:10]

legacy_tfidf_filtered = legacy_tfidf[:, top_10_indices]

In [15]:
joblib.dump(top_10_indices,'thick_col.pkl')

['thick_col.pkl']

In [16]:
import optuna
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

# Define the objective function for optimization
def objective(trial):
    # Define the search space
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1, 10),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'random_strength': trial.suggest_loguniform('random_strength', 0.1, 10),
        'bagging_temperature': trial.suggest_loguniform('bagging_temperature', 0.1, 10)
    }
    
    # Initialize CatBoostClassifier with suggested parameters
    catboost_model = CatBoostClassifier(**params, random_state=42, verbose=0)
    
    # Split the data into train and validation sets
    X_train, X_val, y_train, y_val = train_test_split(legacy_tfidf_filtered, thick_data['target'], test_size=0.2, random_state=42)
    
    # Fit the model
    catboost_model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=10, verbose=False)
    
    # Get validation predictions
    y_pred = catboost_model.predict(X_val)
    
    # Calculate precision, recall, and F1 score for class '1'
    precision = precision_score(y_val, y_pred, pos_label=1)
    recall = recall_score(y_val, y_pred, pos_label=1)
    f1 = f1_score(y_val, y_pred, pos_label=1)
    
    return f1

# Create an Optuna study object
study = optuna.create_study(direction='maximize')

# Optimize the objective function
study.optimize(objective, n_trials=50)

# Get the best parameters and best score
best_params = study.best_params
best_score = study.best_value

print("Best Parameters:", best_params)
print("Best F1 Score for class '1':", best_score)

# Train CatBoostClassifier with the best parameters
best_catboost_model = CatBoostClassifier(**best_params, random_state=42, verbose=0)


[I 2024-02-19 19:45:57,203] A new study created in memory with name: no-name-b02189a0-335e-4802-b255-d06348e6b8cf


C:\Users\jaeyeon\AppData\Local\Temp\ipykernel_18580\2322539660.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
C:\Users\jaeyeon\AppData\Local\Temp\ipykernel_18580\2322539660.py:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1, 10),
C:\Users\jaeyeon\AppData\Local\Temp\ipykernel_18580\2322539660.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'random_strength': trial.suggest_l

Best Parameters: {'iterations': 224, 'learning_rate': 0.016597234621155716, 'depth': 5, 'l2_leaf_reg': 3.550253818036615, 'border_count': 135, 'random_strength': 0.22454693355572314, 'bagging_temperature': 2.117684644858476}
Best F1 Score for class '1': 0.4125874125874126


In [17]:
best_catboost_model.fit(legacy_tfidf_filtered, thick_data['target'])
joblib.dump(best_catboost_model, 'thick_catboost_model.pkl')

['thick_catboost_model.pkl']